In [1]:
import nltk                
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shara\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\shara\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import textstat                  
import openpyxl                 
import string                  
import spacy                  
import re   
import requests                  
import pandas as pd              
from bs4 import BeautifulSoup    
from textblob import TextBlob    

In [3]:
with open(r"C:\Users\shara\Downloads\BlackCoffer/negative-words.txt","r",encoding = "ISO-8859-1") as neg:
    negw = neg.read().split("\n")

In [4]:
with open(r"C:\Users\shara\Downloads\BlackCoffer/positive-words.txt","r") as pos:
    posw = pos.read().split("\n") 

In [5]:
x= print(__name__ == '__main__')

True


In [6]:
import openpyxl

data=[]

def text(): 

    #### This is for fetching the data from given input excel sheet #####
    wbx = openpyxl.load_workbook('Input.xlsx')  
    wsx = wbx['Sheet1']
    
    #### for fetching url from sheet into url variable ##### 
    for i in range (2 , 172):
        urllink = (wsx.cell(row=i, column=2).value)
    
    #pass argument:Headers by passing User-Agent to the request to bypass mod-security error.

        headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}     
        page = requests.get(urllink, headers=headers)
    
    # apply BeautifulSoup 
        soup = BeautifulSoup(page.content, 'html.parser')
        
    # fetch title from link
        title = soup.find('h1',class_="entry-title").text.replace('\n'," ")
    
    #fetch content from link and remove unwanted punctuation
        content = soup.findAll(attrs={'class':'td-post-content'})    
        content = content[0].text.replace('\n'," ")    
        content = content.translate(str.maketrans('', '', string.punctuation)) 
    
    #tokenize data 
        text_tokens = word_tokenize(content)
        
    
    #removing unecessary stopwords
        my_stop_words = stopwords.words('english')
        no_stop_tokens = [word for word in text_tokens if not word in my_stop_words]
    
    #count positive score using positive dictionary 
        pos_count = " ".join ([w for w in no_stop_tokens if w in posw])   
        pos_count=pos_count.split(" ")  
        Positive_score=len(pos_count)
    
    #count negative score using negative dictionary
        neg_count = " ".join ([w for w in no_stop_tokens if w in negw])    
        neg_count=neg_count.split(" ")    
        Negative_score=len(neg_count)
        
    #join the filter data after removing stpowords  
        filter_content = ' '.join(no_stop_tokens)
        
    #words count 
        Word_Count=len(content)
        
    #Avg_Sentence_Length count 
        Avg_Sentence_Length = len(content.replace(' ',''))/len(re.split(r'[?!.]', content))
    
    #calculating fog index using textstat lib
        Fog_Index=(textstat.gunning_fog(content))
    
    #Avg_Number_of_Words_Per_Sentence count
        Avg_Number_of_Words_Per_Sentence = [len(l.split()) for l in re.split(r'[?!.]', content) if l.strip()]
        Avg_Number_of_Words_Per_Sentence=(sum(Avg_Number_of_Words_Per_Sentence)/len(Avg_Number_of_Words_Per_Sentence))
    
        Word_Count=len(content)

        
    #func to calc Complex_Words considering word not ending from "ed" or "es"
        def syllablecount(word):
            coun = 0
            vowels = "AEIOUYaeiouy"
            if word[0] in vowels:
                coun = coun + 1
            for index in range(1, len(word)): 
                    if word[index] in vowels and word[index - 1] not in vowels:
                        coun = coun + 1
                        if word.endswith("es"or "ed"):
                            coun = coun - 1
            if coun == 0:
                coun = coun + 1
            return coun
        Complex_Words = syllablecount(content)

    
    #func to calc proper noun in article with help of tagging from nltk lib
        def ProperNounExtractor(text):
            cou = 0
            sentences = nltk.sent_tokenize(text)
            for sentence in sentences:
                words = nltk.word_tokenize(sentence)
                tagged = nltk.pos_tag(words)
                for (word, tag) in tagged:
                    if tag == 'PRP': # If the word is a proper noun
                        cou = cou + 1 
        
            return(cou) 
        Personal_Pronouns=ProperNounExtractor(content)
    

    #function for sentiment analysis
        def sentiment_analysis(text):
            sentiment = TextBlob(text).sentiment
            return (sentiment.polarity)
    
        polarity=sentiment_analysis(content)
  
        def sentiment_analysis(text):
            sentiment = TextBlob(text).sentiment
            return (sentiment.subjectivity)
    
        subjectivity=sentiment_analysis(content)
        
        
    #method to count average syllable count
        word=content.replace(' ','')
        syllable_count = 0
        for w in word:
            
            if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
                syllable_count=syllable_count+1

        Syllable_Per_Word=(syllable_count/len(content.split()))
        
    # calculate avg word length
        Average_Word_Length=len(content.replace(' ',''))/len(content.split())
        
    # calculate % of complex word
        Percentage_of_Complex_Word = Complex_Words / Word_Count * 100
    
        data.insert(i,[urllink,Positive_score, Negative_score, polarity,subjectivity, Avg_Sentence_Length,Percentage_of_Complex_Word,Fog_Index, Avg_Number_of_Words_Per_Sentence , Complex_Words, Word_Count,Syllable_Per_Word, Personal_Pronouns, Average_Word_Length])
 
 
if __name__ == '__main__' :
    text()
        
df = pd.DataFrame(data,columns=['url','Positive_score','Negative_score','polarity','subjectivity', 'Avg_Sentence_Length','Percentage_of_Complex_Word', 'Fog_Index', 'Avg_Number_of_Words_Per_Sentence' , 'Complex_Words', 'Word_Count', 'Syllable_Per_Word','Personal_Pronouns', 'Average_Word_Length'])

In [7]:
df.to_csv('linkd.csv')
df

,url,Positive_score,Negative_score,polarity,subjectivity,Avg_Sentence_Length,Percentage_of_Complex_Word,Fog_Index,Avg_Number_of_Words_Per_Sentence,Complex_Words,Word_Count,Syllable_Per_Word,Personal_Pronouns,Average_Word_Length
0,https://insights.blackcoffer.com/how-is-login-...,16,9,0.143040,0.478514,3673.0,29.160967,289.41,712.0,1279,4386,2.109551,12,5.158708
1,https://insights.blackcoffer.com/how-does-ai-h...,27,3,0.177208,0.493776,3357.0,30.125313,257.97,632.0,1202,3990,2.118671,18,5.311709
2,https://insights.blackcoffer.com/ai-and-its-im...,76,20,0.141910,0.539460,9654.0,29.886562,726.17,1804.0,3425,11460,2.148559,44,5.351441
3,https://insights.blackcoffer.com/how-do-deep-l...,14,1,0.077056,0.458329,2306.0,30.182482,179.39,433.0,827,2740,2.166282,6,5.325635
4,https://insights.blackcoffer.com/how-artificia...,51,13,0.063581,0.471385,6191.0,30.504338,493.42,1222.0,2250,7376,2.085925,43,5.066285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,https://insights.blackcoffer.com/role-big-data...,41,18,0.130195,0.420271,7700.0,30.315858,609.48,1514.0,2793,9213,2.058785,32,5.085865
166,https://insights.blackcoffer.com/sales-forecas...,36,19,0.102483,0.481812,5254.0,30.347938,384.24,949.0,1884,6208,2.163330,10,5.536354
167,https://insights.blackcoffer.com/detect-data-e...,17,40,0.050360,0.486082,5253.0,30.990672,389.71,963.0,1927,6218,2.229491,17,5.454829
168,https://insights.blackcoffer.com/data-exfiltra...,2,7,0.057445,0.480637,2878.0,30.420992,238.10,586.0,1055,3468,2.003413,20,4.911263
